In [70]:
import pandas as pd
import math

In [71]:
def calcular_quantidade_log(x):
    if x == 0 or x == 1:
        return math.log(1.01)
    else:
        return math.log(x)

In [72]:
pd.options.mode.chained_assignment = None

# BASE RFV

In [73]:
def criar_localidade(base):

  base['localidade'] = base.apply(
    lambda row: row['bairro 2'] if row['população_IBGE'] >= 50000 else row['Município'], axis=1)

In [74]:
def transformar_em_datetime(base):
  base['Data Fato'] = pd.to_datetime(base['Data Fato'])

In [75]:
def criar_indice(bairro, base, nome_base, peso):

  planilha_filtro_bairro_geral = base[base['Território'] == bairro]

  planilha_filtro_bairro = planilha_filtro_bairro_geral

  if planilha_filtro_bairro.empty:
      return None

  total_por_bairro_e_mes = planilha_filtro_bairro.groupby([planilha_filtro_bairro['Território'], pd.Grouper(key='Data Fato', freq='M')])['qtd'].sum().reset_index()


  bairros = planilha_filtro_bairro['Território'].unique()
  meses = pd.date_range(start=planilha['Data Fato'].min(), end=planilha['Data Fato'].max(), freq='M')
  combinacoes = pd.MultiIndex.from_product([bairros, meses], names=['Território', 'Data Fato'])
  df_combinado = pd.DataFrame(index=combinacoes).reset_index()


  total_por_bairro_e_mes = pd.merge(df_combinado, total_por_bairro_e_mes, how='outer', on=['Território', 'Data Fato'])
  total_por_bairro_e_mes['qtd'] = total_por_bairro_e_mes['qtd'].fillna(0)

  total_por_bairro_e_mes['quantidade_log'] = total_por_bairro_e_mes['qtd'].apply(calcular_quantidade_log)

  total_por_bairro_e_mes['qtd'] = total_por_bairro_e_mes['qtd'].astype(int)


  quantidade_log_maximo = total_por_bairro_e_mes['quantidade_log'].max()
  maximo_ajustado = quantidade_log_maximo * 1.05
  quantidade_log_minimo = total_por_bairro_e_mes['quantidade_log'].min()
  minimo_ajustado = quantidade_log_minimo * 0.95
  diferenca_max_min = maximo_ajustado - minimo_ajustado

  #peso = peso

  total_por_bairro_e_mes[f'indice_{nome_base}'] = (1-(maximo_ajustado-total_por_bairro_e_mes['quantidade_log'])/diferenca_max_min)*peso
  municipio = planilha_filtro_bairro['Município'].unique()[0]
  total_por_bairro_e_mes['Município'] = f'{municipio}'
  total_por_bairro_e_mes.rename(columns={'qtd': f'#{nome_base}', 'quantidade_log': f'#{nome_base}_log'}, inplace=True)


  return total_por_bairro_e_mes

In [76]:
def run_base(municipio, nome_base, peso):
    df = planilha.loc[planilha['Município'] == municipio]
    todos_bairros = df['Território'].unique()
    lista_bairros = todos_bairros
    base = df

    df_final = pd.DataFrame()
    for bairro in lista_bairros:
      bairro_mes = criar_indice(bairro, base, nome_base, peso)
      df_final = pd.concat([df_final, bairro_mes], ignore_index=True)

    col_municipio = df_final.pop("Município")
    df_final.insert(0, "Município", col_municipio)

    return df_final

#def run_rfv(municipio):
#    df = planilha.loc[planilha['Município'] == municipio]
#    todos_bairros = df['Território'].unique()
#    lista_bairros = todos_bairros
#    base = df

#    df_rfv_final = pd.DataFrame()
#    for bairro in lista_bairros:
#      rfv_bairro_mes = criar_indice_rfv(bairro, base, 'rfv', 0.4)
#      df_rfv_final = pd.concat([df_rfv_final, rfv_bairro_mes], ignore_index=True)

#    col_municipio = df_rfv_final.pop("Município")
#    df_rfv_final.insert(0, "Município", col_municipio)

#    return df_rfv_final

In [77]:
planilha_geral = pd.read_excel('ocorrencias.xlsx', sheet_name='BASE_TRATADA')

# BASE RFV

In [78]:
planilha = planilha_geral[planilha_geral['Base'] == 'RFV']
lista_municipios = planilha['Município'].unique()
#print(len(lista_municipios))

transformar_em_datetime(planilha)

df_rfv_final = pd.DataFrame()
for municipio in lista_municipios:
    #print(municipio)
    try:
        df_rfv = run_base(municipio, 'rfv', 0.4)

        df_rfv_final = pd.concat([df_rfv_final, df_rfv], ignore_index=True)
    except Exception as e:
      print(e)
      #print('Deu erro!!')
      continue

In [79]:
df_final.to_csv('output/base_rfv.csv', sep=';', encoding='latin', index=False)

# BASE VIOLÊNCIA

In [80]:
planilha_violencia = planilha_geral[planilha_geral['Base'] == 'VIOLÊNCIA']
planilha = planilha_violencia[planilha_violencia['Sexo'] == 'FEMININO']
lista_municipios = planilha['Município'].unique()
#print(len(lista_municipios))

transformar_em_datetime(planilha)

df_violencia_final = pd.DataFrame()
for municipio in lista_municipios:
    #print(municipio)
    try:
        df_violencia = run_base(municipio, 'violencia', 0.8)

        df_violencia_final = pd.concat([df_violencia_final, df_violencia], ignore_index=True)
    except Exception as e:
      print(e)
      #print('Deu erro!!')
      continue

In [81]:
df_final.to_csv('output/base_violencia.csv', sep=';', encoding='latin', index=False)

# BASE MVI

In [82]:
planilha = planilha_geral[planilha_geral['Base'] == 'MVI']
lista_municipios = planilha['Município'].unique()
#print(len(lista_municipios))

transformar_em_datetime(planilha)

df_mvi_final = pd.DataFrame()
for municipio in lista_municipios:
    #print(municipio)
    try:
        df_mvi = run_base(municipio, 'mvi', 0.6)

        df_mvi_final = pd.concat([df_mvi_final, df_mvi], ignore_index=True)
    except Exception as e:
      print(e)
      #print('Deu erro!!')
      continue

In [83]:
df_final.to_csv('output/base_mvi.csv', sep=';', encoding='latin', index=False)

# ARRUMANDO TUDO

In [84]:
bo_por_bairro_e_mes = planilha_geral.groupby([planilha_geral['Município'], planilha_geral['Território'], pd.Grouper(key='Data Fato', freq='M')])['qtd'].sum().reset_index()

bo_por_bairro_e_mes.rename(columns={'qtd': '#bo'}, inplace=True)
#print(bo_por_bairro_e_mes)

df_final = pd.DataFrame()
#df_rfv_final = pd.read_csv('ETL/output/base_rfv.csv', sep=';', encoding='latin')
#df_violencia_final = pd.read_csv('ETL/output/base_violencia.csv', sep=';', encoding='latin')
#df_mvi_final = pd.read_csv('ETL/output/base_mvi.csv', sep=';', encoding='latin')

combinado = pd.merge(df_rfv_final, df_violencia_final,
                     left_on=['Município', 'Território', 'Data Fato'],
                     right_on=['Município', 'Território', 'Data Fato'],
                     how='outer')
combinado2 = pd.merge(combinado, df_mvi_final,
                     left_on=['Município', 'Território', 'Data Fato'],
                     right_on=['Município', 'Território', 'Data Fato'],
                     how='outer')
combinado2['Data Fato']=combinado2['Data Fato'].astype('datetime64[ns]')
bo_por_bairro_e_mes['Data Fato']=bo_por_bairro_e_mes['Data Fato'].astype('datetime64[ns]')


combinado3 = pd.merge(combinado2, bo_por_bairro_e_mes,
                     left_on=['Município', 'Território', 'Data Fato'],
                     right_on=['Município', 'Território', 'Data Fato'],
                     how='left')


combinado3['#bo'] = combinado3['#bo'].fillna(0)



df_final = pd.concat([df_final, combinado3], ignore_index=True)
#Município_x;Território;Data Fato;qtd_x;quantidade_log_x;indice_rfv;Município_y;qtd_y;quantidade_log_y;indice_violencia;Município;qtd;quantidade_log;indice_cvli
#df_final.pop('Município_x')
#df_final.pop('Município_y')







df_final.to_csv('output/indice_violencia.csv', sep=';', encoding='latin', index=False)

In [85]:
df_final

,Município,Território,Data Fato,#rfv,#rfv_log,indice_rfv,#violencia,#violencia_log,indice_violencia,#mvi,#mvi_log,indice_mvi,#bo
0,RECIFE,TORROES (RECIFE),2020-01-31,4.0,1.386294,0.239697,23.0,3.135494,0.383932,4.0,1.386294,0.380000,64.0
1,RECIFE,TORROES (RECIFE),2020-02-29,3.0,1.098612,0.189614,15.0,2.708050,0.192549,0.0,0.009950,0.000137,53.0
2,RECIFE,TORROES (RECIFE),2020-03-31,2.0,0.693147,0.119026,14.0,2.639057,0.161659,2.0,0.693147,0.188695,41.0
3,RECIFE,TORROES (RECIFE),2020-04-30,9.0,2.197225,0.380874,11.0,2.397895,0.053681,6.0,1.791759,0.491906,46.0
4,RECIFE,TORROES (RECIFE),2020-05-31,3.0,1.098612,0.189614,11.0,2.397895,0.053681,0.0,0.009950,0.000137,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110335,QUIXABA,ZONA RURAL (QUIXABA) (QUIXABA),2022-08-31,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.009950,0.300000,0.0
110336,QUIXABA,ZONA RURAL (QUIXABA) (QUIXABA),2022-09-30,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.009950,0.300000,0.0
110337,QUIXABA,ZONA RURAL (QUIXABA) (QUIXABA),2022-10-31,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.009950,0.300000,0.0
110338,QUIXABA,ZONA RURAL (QUIXABA) (QUIXABA),2022-11-30,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.009950,0.300000,0.0
